In [ ]:
import cv2
import dlib
import numpy as np

In [ ]:
faceDet = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
def FaceExtractor(img):
    global cropface
    Gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceDet.detectMultiScale(Gray,1.3,2)
    if faces is None:
        return None;
   # for face in faces:
   #     x = face.left()
   #     y = face.top()
   #     w = face.right() - x
   #     h = face.bottom() - y
   #     FaceCrop = img[y:y+h,x:x+h]
   #     return FaceCrop
    for (x,y,w,h) in faces:
        global cropface
        cropface = img[y:y+h,x:x+w]
    return cropface

In [ ]:
cnt = 0
cap = cv2.VideoCapture(0)
while True:
    ret, img = cap.read()
    if FaceExtractor(img) is None:
        print("No face found")
    else:
        a= FaceExtractor(img)
        cv2.imshow('Hi',a)
        cnt+=1
        file_name_path = '/Users/kritiksachdeva/MyProject/FaceRecognition/Myface/'+str(cnt)+'.jpg'
        cv2.imwrite(file_name_path,a)
    if cv2.waitKey(1) == 13 or cnt == 100:
        break
    
cap.release()
cv2.destroyAllWindows()
print("collecting Samples is complete")

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
data_path = '/Users/kritiksachdeva/MyProject/FaceRecognition/Myface/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

Training_data, Labels = [], []

for i,files in enumerate(onlyfiles):
    img_path = data_path + onlyfiles[i]
    images = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    Training_data.append(np.asarray(images,dtype=np.uint8))
    Labels.append(i)
Labels = np.asanyarray(Labels)

model = cv2.face.createLBPHFaceRecognizer()
model.train(np.asarray(Training_data),np.asarray(Labels))
print("Model Trained Sucessfully")

In [ ]:
def faceExtract(img):
    global ro
    Gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceDet.detectMultiScale(Gray,1.3,2)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        global ro 
        ro = img[y:y+h,x:x+w]
    return ro

cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    #print(ret)
    cropFace = faceExtract(image)
    try:
        face = cv2.cvtColor(cropFace,cv2.COLOR_BGR2GRAY)
        results = model.predict(face)
        print(results)
        if results[1] < 100:
            confidence = int( 100 * (1 - (results[1])/100) )
            display_string = str(confidence) + '% Confident it is User'

            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

            if confidence > 35:
                cv2.putText(image, "Hey aditya", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                threshold( image, thr, 100,255,THRESH_BINARY )
                cv2.imshow('Face Recognition', image )
                break
            else:
                cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.imshow('Face Recognition', image )
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()